In [3]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit  
from langchain_community.utilities.sql_database import SQLDatabase  
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

In [4]:
load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [5]:
db = SQLDatabase.from_uri("sqlite:///Report_Card.db")
db

In [6]:
llm =  ChatGroq(model= 'llama3')
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [7]:
for tool in toolkit.get_tools():
    print(tool.name)

sql_db_query
sql_db_schema
sql_db_list_tables
sql_db_query_checker


In [ ]:
from langchain_core.tools import tool
@tool

def db_query_tool(query:str) -> str:
    '''
    Execute a SQL Query against the database and return the result.
    If the query is invalid and returns no result, an error message will be returned.
    In case of an error, the user is advised to rewrite the query and try again.
    '''
    result = db.run_no_throw(query)
    if not result:
        return 'Error!! Query failed please rewrite the query.'
    else:
        return result